<a href="https://colab.research.google.com/github/frikel12/Machine-Learning-and-Deep-Learningproject/blob/main/Features_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <h1>Ingénierie des caractéristiques <br> (Features engineering)</h1></center>


<h1>Ingénierie de caractéristiques</h1>
L'Ingénierie de caractéristiques consiste en le traitement des caractéristiques dans le but d'améliorer le comportement du modéle de classification. <br>
Plusieurs étapes peuvent être utilisées (pas toutes utilisables pour l’image mining)

* Valeurs manquantes (Non rencotrés en Image Mining)
* Valeurs aberrantes
* Transformation logarithmique
* Encodage unique (Concerne uniquement les étiquettes en Image Mining)
* Mise en échelle
* Sélection des caractéristiques


<h1> Implémentation </h1>

<h2> Préparation de l'environnement</h2>
Nous allons travailler dans cet atelier sur des caractéristiques déja extraites et enregistrées sur Google Drive  (Voir atelier tranfer Learning).

In [ ]:
#from keras.preprocessing import image
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<h2> Chargement de la dataset</h2>
Pour tester et appliquer l'ingénierie de caractéristiques, nous allons utiliser des caractéristiques déjà extraites utilisant une architecture Deep Learning VGG16 de la base d'image Corel utilisée dans l'atelier de classification supervisée.
Pour télécharger le fichier de caractéristiques et les étiquettes cliquer sur les liens ci-dessous :

[Features](https://drive.google.com/file/d/105zEqcYD1Deuzy5NM5dOPqnXDETUAfVt/view?usp=share_link)


[Labels](https://drive.google.com/file/d/1aggw9QXm9ao7CEmLa4ign9xi2zovUPnB/view?usp=drivesdk)

<h3>Charger les caractéristiques et les étiquettes.</h3>

In [ ]:
# Load features and labels
import pickle
fetauresPath='/content/drive/MyDrive/Datasets/Image Mining/'
X = pickle.load( open( fetauresPath+"features_vgg16", "rb" ) )
y = pickle.load( open( fetauresPath+"labels", "rb" ) )

import numpy as np
print(np.array(X).shape, np.array(y).shape)

(490, 1000) (490,)


Pour chacune des étapes de l'ingénierie de caractéristiques, nous allons évaluer à chaque fois la classification afin de voir l'impact.
Compléter l'implémentation de la fonction de classification. Utiliser un algorithme de classification de votre choix.
La méthode doit afficher le taux de classification (accuracy) ainsi que le temps d'exécution de classification.

In [ ]:
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

def classify(X,y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  model = svm.SVC(kernel='rbf')
  model.fit(X_train, y_train)

  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)

  return accuracy



Classification utilisant les caractéristiques initiales sans manipulation

In [ ]:
# Afficher l'accuracy
print("accuracy : ", round(classify(X,y), 2))

accuracy :  0.94


<h1>Mise en échelle</h1>
Dans la plupart des cas, les caractéristiques de type numérique ne sont pas dans la même plage d’intervalle.
Exemple : les données sur les âges et les salaires différent et ne n’ont pas la même fourchette.
Pour un algorithme d’apprentissage automatique, les valeurs qui ne changent pas beaucoup même sur différentes échelles vont être très influentes en termes de classification.
Les algorithmes de classification basés sur le voisinage ou distance sont très sensible à la mise en échelle ; comme le KNN, K-means, …
Pour mettre en échelle les données nous avons deux solutions :

1.   Normalisation : min-max normalization
2.   Standardisation : z-score normalization



<h2> Standarization: z-score normalization</h2>
<center>z=(X-µ)/σ

X est l'ensemble d'apprentissage, µ est la moyenne et  σ est l’écart type
</center>
Le z-score met en échelle les valeurs en prenant en compte l’écart type (déviation standard). <br>
le z-score réduit l’effet des données aberrantes

In [ ]:
"""Standarization
Standardize features by removing the mean and scaling to unit variance.
z = (x - u) / s
where u is the mean of the training samples or zero if with_mean=False
s is the standard deviation of the training samples or one if with_std=False.
"""
# Xzscore est la matrice de caractéristiques après standardisation
mean = np.mean(X, axis=0)
std = np.std(X, axis=0)
Xzscore = (X - mean)/std
#print(np.array(Xzscore).shape)
print("accuracy : ", round(classify(Xzscore,y), 2))

accuracy :  0.95


<h2>Normalization: min-max normalization</h2>
<center>Xnorm=(X-Xmin)/(Xmax -Xmin)</center>
Permet de mettre en échelle toutes les valeurs des caractéristiques dans une plage fixe entre 0 et 1.<br>
L'inconvénient de la normalization par min-max est qu'elle augmente les effets des valeurs aberrantes.<br>
Ainsi, avant la normalisation, il est recommandé de traiter les valeurs aberrantes.



In [ ]:
"""Noramlization
Xnorm=(X-Xmin)/(Xmax -Xmin).
"""
# Xminmax est la matrice de caractéristiques après normalisation

Xminmax=(X - X.min(axis=0))/(X.max(axis=0) - X.min(axis=0))

#print(np.array(Xminmax).shape)
print("accuracy : ", round(classify(Xminmax,y), 2))


accuracy :  0.97


<h2> Transformation logarithmique</h2>
Pour éliminer les données aberrantes, une des techniques les plus utilisées est la transformation logarithmique.

*	Traiter des données biaisées et, après transformation, la distribution devient plus proche de la normale.
*	Réduire l’ordre de grandeur des données. Exemple : la différence de taille n’est pas de la même grandeur que la différence d'âges
*	Réduit aussi l'effet des valeurs aberrantes grâce à la normalisation des différences d'amplitude

NB : il ne faut pas que les données soient négatives





In [ ]:
# Transformation logarithmique
# Xlog est la matrice de caractéristiques après transformation logarithmique
Xlog = np.log(X)
# Classification utilisant les caractéristiques aprés tranformation logarithmique
print("accuracy : ", round(classify(Xlog,y), 2))


accuracy :  0.97


Par la suite nous allons procéder a la mise en échelle des données après transformation logarithmique

In [ ]:
# Normalization aprés TL

Xminmax_Log = (Xlog - Xlog.min(axis=0))/(Xlog.max(axis=0) - Xlog.min(axis=0))
print("accuracy : ", round(classify(Xminmax_Log,y), 2))


accuracy :  0.97


In [ ]:
# Standarization + TL
mean = np.mean(Xlog, axis=0)
std = np.std(Xlog, axis=0)
Xzscore_Log = (Xlog - mean)/std
print("accuracy : ", round(classify(Xzscore_Log,y), 2))


accuracy :  0.98


<h1>Sélection de caractéristiques (features selection)</h1>

<h2> Suppression des caractéristiques à faible variance</h2>
Suppression des caractéristiques à faible variance (Removing features with low variance) est une méthode de sélection (élimination) basée sur le filtrage.<br>
On peut la considérer comme une méthode de nettoyage de caractéristique qui  élémine toutes les caractéristiques dont la variance n'atteint pas un certain seuil.<br>
Par défaut, elle supprime toutes les caractéristiques à variance nulle, c'est-à-dire les caractéristiques qui ont la même valeur dans tous les échantillons.

In [ ]:
# Nous allons utiliser les caractéristiques après normalisation Min-Max
# Xvth est la matrice de caractéristiques après suppression des caractéristiques (après normalisation Min-Max)  à faible variance
from sklearn.feature_selection import VarianceThreshold

vth = VarianceThreshold()
Xvth = vth.fit_transform(Xminmax)
print("accuracy : ", round(classify(Xvth,y), 2))


accuracy :  0.97


<h2>chi2</h2>
C’est un algorithme de sélection des caractéristiques qui appartient à la famille des algorithmes de filtrage basé sur la statistique 𝜒2. Cette méthode mesure l’écart à l’indépendance entre une caractéristique et une classe. Elle commence par un niveau de signification élevé pour toutes les caractéristiques pour la discrétisation et chaque caractéristique est triée en fonction de ses valeurs.

In [ ]:
#Xchi2 est la matrice de caractéristiques après sélection de 100 caractéristiques utilisant chi2
from sklearn.feature_selection import SelectKBest, chi2

chi2 = SelectKBest(chi2, k=100)
Xchi2 = chi2.fit_transform(Xminmax, y)
print("accuracy : ", round(classify(Xchi2,y), 2))


accuracy :  0.95


<h2>Recursive Feature Elimination (RFE)</h2>
C’est une méthode de cartographie basée sur l'idée à plusieurs reprises, construire un modèle et choisir le meilleur ou le pire performant. Cette méthode qui appartient aux méthodes de filtrage est souvent utilisée comme étape de prétraitement pour les méthodes intégrée (souvent avec l’algorithme de classification SVM) afin de la généraliser à des grandes masses de données


In [ ]:
# Xrfe est la matrice de caractéristiques après sélection de 100 caractéristiques utilisant RFE
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

estimator = LogisticRegression()

rfe = RFE(estimator, n_features_to_select=100)
Xrfel = rfe.fit_transform(Xminmax, y)

print("accuracy : ", round(classify(Xrfel,y), 2))


accuracy :  0.98


Essayer d'identifier le nombre minimal de caractéristiques à utiliser pour obtenir le taux maximal de classification.

In [ ]:
# Code pour calculer le nombre minimal de caractéristiques à utiliser pour obtenir le taux maximal de classification
rfe = RFE(estimator, n_features_to_select=82)
Xrfel = rfe.fit_transform(Xminmax, y)

print("accuracy : ", round(classify(Xrfel,y), 2))


accuracy :  0.98


<h1>Relief</h1>
Tente de déterminer le plus proche voisin d'un certain nombre d'échantillons sélectionnés au hasard à partir de l'ensemble de données. Pour chaque échantillon sélectionné, les valeurs des caractéristiques sont comparées à ceux des voisins les plus proches et les scores pour chaque caractéristique sont mis à jour. L'idée est d'estimer la qualité des attributs en fonction de la qualité de leurs valeurs et faire la distinction entre des échantillons proches les uns des autres

In [ ]:
#pour utiliser la technique de selection Relief, on peut se servir de la bibliotheque skrebate
!pip install skrebate

  Preparing metadata (setup.py) ... done
  Created wheel for skrebate: filename=skrebate-0.62-py3-none-any.whl size=29253 sha256=a35e12c7c43914d5073af7e1ff441f9f06fbf15f4776eb5c0f630a4c3ea7c694
  Stored in directory: /root/.cache/pip/wheels/dd/67/40/683074a684607162bd0e34dcf7ccdfcab5861c3b2a83286f3a
Successfully built skrebate


In [ ]:
# XRelief est la matrice de caractéristiques après sélection de 100 caractéristiques utilisant Relief
from skrebate import ReliefF

relief = ReliefF(n_features_to_select=100)
XRelief = relief.fit_transform(Xminmax, y)
print("accuracy : ", round(classify(XRelief,y), 2))


accuracy :  0.96


Essayer d'identifier le nombre minimal de caractéristiques à utiliser pour obtenir le taux maximal de classification.

In [ ]:
# Code pour calculer le nombre minimal de caractéristiques à utiliser pour obtenir le taux maximal de classification
relief = ReliefF(n_features_to_select=40)
XRelief = relief.fit_transform(Xminmax, y)
print("accuracy : ", round(classify(XRelief,y), 2))


accuracy :  0.96


<h1>Réduction de la dimensionnalité </h1>
La réduction de la dimensionnalité transforme les caractéristiques en une dimension inférieure. Elle peut être considérée comme étant une méthode de  Selection ou de création (extraction) de caractéristiques où nous dérivons des informations à partir de l’ensemble de caractéristiques de base pour construire un nouveau sous espace de caractéristiques. <br>
Ls approches de réduction de dimensionnalité les plus connues sont: PCA (Principal Component Analysis; Analyse en Composantes Principales (ACP)) et ICA (Independent Component Analysis, Analyse en Composantes Independantes (ACI))





<h2> ACP </h2>

In [ ]:
# Xpca est la matrice de caractéristiques après réduction de la dimensionalité utilisant l'ACP
# Essayer de changer la taille de réduction de l'ACP afin d'obtenir le meilleur taux de classification
from sklearn.decomposition import PCA

pca = PCA(n_components=120)
Xpca = pca.fit_transform(Xminmax)
print(np.array(Xpca).shape)
print("accuracy : ", round(classify(Xpca,y), 2))


(490, 120)
accuracy :  0.97


<h2> ACI </h2>

In [ ]:
# Xica est la matrice de caractéristiques après réduction de la dimensionalité utilisant l'ACI
# Essayer de changer la taille de réduction de l'ACI afin d'obtenir le meilleur taux de classification
from sklearn.decomposition import FastICA

ica = FastICA(n_components=90)
Xica = ica.fit_transform(Xminmax)
print(np.array(Xica).shape)
print("accuracy : ", round(classify(Xica,y), 2))


/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


(490, 90)
accuracy :  0.97
